# 오징어 게임(달고나 게임 만들기)

In [1]:
import cv2
import numpy as np

In [2]:
def dalgona_game():
    global isDragging
    img = cv2.imread("./img/star.jpg")
    rows, cols=img[:2]

    sample_star=np.full_like(img, (255,255,255)) #기본 달고나의 컨투어
    user_star=np.full_like(img, (255,255,255)) #사용자가 쓰는 곳
    #달고나 표면을 흐릿하게..피라미드 평균 시프트 적용
    mean = cv2.pyrMeanShiftFiltering(img, 20, 30) #첫 이미지..
    mean1=mean.copy()
    gray = cv2.cvtColor(mean1, cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray, (3,3), 0)
    edges = cv2.Canny(gray, 100, 370)

    thresh_star = cv2.adaptiveThreshold(edges, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 9, 9)
    cntr_stars, _ = cv2.findContours(thresh_star, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #for i in range(len(cntr_stars)):
    cntr_star = cntr_stars[4]
    cv2.drawContours(mean1, [cntr_star], -1, (0,255,0), 10)
    cv2.drawContours(sample_star, [cntr_star], -1, (0,0,0), 10)

    #사용자 달고나 게임 진행하기
    isDragging = False
    def onMouse(event, x, y, flags, params):
        global isDragging
        if event == cv2.EVENT_LBUTTONDOWN:
            isDragging = True
        elif event == cv2.EVENT_MOUSEMOVE:
            if  isDragging:
                cv2.circle(mean, (x,y), 5, (255,255,255),-1)
                cv2.circle(user_star, (x,y), 5, (0,0,0),-1)
                cv2.imshow("mean", mean)
        elif event == cv2.EVENT_LBUTTONUP:
            isDragging = False
            if cv2.waitKey(0) == 13: # 엔터키 누르면
                user_gray = cv2.cvtColor(user_star, cv2.COLOR_BGR2GRAY)
                sample_gray = cv2.cvtColor(sample_star, cv2.COLOR_BGR2GRAY)
                ret, userTh = cv2.threshold(user_gray, 127, 255, cv2.THRESH_BINARY_INV)
                ret, sampleTh = cv2.threshold(sample_gray, 127, 255, cv2.THRESH_BINARY_INV)

                user_cnt, _ = cv2.findContours(userTh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                sample_cnt, _ = cv2.findContours(sampleTh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                #매치율 확인하기
                matchs=[]
                for contr in user_cnt:
                    match=cv2.matchShapes(sample_cnt[0], contr, cv2.CONTOURS_MATCH_I2, 0.0)
                    matchs.append((match, contr))
                    #print(match)
                    #print(tuple(contr[0][0]))
                    if match<=0.05:
                        cv2.putText(mean, "%.2f Success!!"%match, (30,150), cv2.FONT_HERSHEY_PLAIN, 2, (237,10,63),2)
                    else: 
                        cv2.putText(mean, "%.2f Game Fail!!"%match, (30,150), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0),2)
                cv2.imshow("mean", mean)

    #게임 스타트
    cv2.imshow("mean", mean)
    cv2.setMouseCallback("mean", onMouse)
    #cv2.imshow("mean1", mean1)
    #cv2.imshow("bin", sample_star)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 숨은 그림 찾기(윌리를 찾아라!)_ 사용자 월리 입력

In [3]:
#특정 부분의 그림을 볼록형식으로 만들어서 틀린 그림을 생성함(roi 쓴것..)
def wally_input(): 
    global dst, img2, cx, cy
    img = cv2.imread("./img/santa.jfif")
    img2=img.copy()

    wally=cv2.imread("./img/wally.png", cv2.IMREAD_UNCHANGED)
    #윌리 줄이기.
    height, width = wally.shape[:2]
    dst = cv2.resize(wally, (int(height*0.05), int(width*0.05)), interpolation=cv2.INTER_AREA)
    cx, xy = 0, 0
    def onMouse(event, x, y, flags, param):
        global dst, img2, cx, cy
        if event == cv2.EVENT_LBUTTONDOWN:
            #이미지 합성하기
            _, mask = cv2.threshold(dst[:,:,3], 1, 255, cv2.THRESH_BINARY)
            mask_inv = cv2.bitwise_not(mask)

            dst = cv2.cvtColor(dst, cv2.COLOR_BGRA2BGR)
            h, w=dst.shape[:2]
            roi = img2[y:y+h, x:x+w] #x좌표 y좌표
            #윌리 좌표
            cx = int((x+(x+w))/2)
            cy = int((y+(y+h))/2)
            print(cx,cy)

            masked_dst=cv2.bitwise_and(dst, dst, mask=mask)
            masked_img = cv2.bitwise_and(roi, roi, mask = mask_inv)

            added = masked_dst + masked_img
            img2[y:y+h, x:x+w]=added
            cv2.imshow("img1", img2)
            cv2.imwrite("./img/santa_wally.png", img2)

    #cv2.imshow("img", dst)
    cv2.imshow("img1", img2)
    cv2.setMouseCallback("img1", onMouse)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 숨은 그림 찾기(윌리를 찾아라!)_ 사용자 월리 찾기

In [4]:
def wally_find(): 
    global cx, cy, user_img, count, cuo, a
    user_img = cv2.imread("./img/santa_wally.png")
    user_img_h, user_img_w=user_img.shape[:2]
    #print(user_img_h, user_img_w)
    count=0
    cuo=0
    a=5
    def onSearch(event, x, y, flags, param):
        global cx, cy, user_img, count, cuo, a
        if event==cv2.EVENT_LBUTTONDOWN:
            count+=1
            if abs(x-cx)<20 and abs(y-cy)<20: #위치가 가까우면 그냥..pass
                #cv2.circle(user_img, (x, y), 10, (0,255,0), 3)
                cv2.circle(user_img, (cx, cy), 10, (0,255,255), 3)
                succeed_rate = ((count-cuo)/count) *100
                #cv2.rectangle(user_img, (50,100),(430,170), (255,255,255),-1)
                cv2.putText(user_img, "Succeed! {0:.1f}%".format(succeed_rate), (50, 150), cv2.FONT_HERSHEY_COMPLEX_SMALL,2, (255,0,0),2)
                cv2.imshow("user_img", user_img)
            else:
                cuo+=1
                cv2.line(user_img, (x-a,y-a),(x+a,y+a),(0,0,255),3)
                cv2.line(user_img, (x-a,y+a),(x+a,y-a),(0,0,255),3)
                cv2.imshow("user_img", user_img)

    cv2.imshow("user_img", user_img)
    cv2.setMouseCallback("user_img", onSearch)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 컬러링 게임

In [5]:
def coloring_game():
    global newVal, mask, imthres
    win_name = 'coloring'
    win_trackbar = 'trackbar'

    img = cv2.imread('./img/coloring1.png', cv2.COLOR_BGR2GRAY)
    trackbar = cv2.imread('./img/blank_500.jpg')
    trackbar = trackbar[:100,:300]

    # 외곽선을 뚜렷하게 하기 위해
    ret, imthres = cv2.threshold(img, 30, 255, cv2.THRESH_BINARY) 

    cv2.imshow(win_name,imthres)
    cv2.imshow(win_trackbar,trackbar)

    rows, cols = imthres.shape[:2]
    mask = np.zeros((rows+2, cols+2), np.uint8)
    loDiff, upDiff = (16,16,16), (30,30,30)
    newVal = (255,255,255)
    def onChange(x):
        global newVal
        trackbar[:] = [newVal[0],newVal[1],newVal[2]]
        cv2.imshow(win_trackbar,trackbar)

    def onMouse(event, x, y, flags, param):
        global mask, imthres, newVal
        if event == cv2.EVENT_LBUTTONDOWN:
            seed = (x,y)
            retval = cv2.floodFill(imthres, mask, seed, newVal, loDiff, upDiff)
            cv2.imshow(win_name,imthres)

    cv2.createTrackbar('R', win_trackbar, 255, 255, onChange)
    cv2.createTrackbar('G', win_trackbar, 255, 255, onChange)
    cv2.createTrackbar('B', win_trackbar, 255, 255, onChange)

    while (True):
        cv2.setMouseCallback(win_name, onMouse)
        r = cv2.getTrackbarPos('R', win_trackbar)
        g = cv2.getTrackbarPos('G', win_trackbar)
        b = cv2.getTrackbarPos('B', win_trackbar)
        newVal = (b, g, r)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cv2.destroyAllWindows()

# 도형맞추기 게임

In [6]:
def find_rectangle():
    global img_draw, isDragging, now, cnt, img
    img = cv2.imread('./img/blank_500.jpg')
    pts1 = np.array([[255,100],[205,180],[305,180]],dtype=np.int32)
    pts2 = np.array([[255,325],[241,353],[205,361],[231,380],[223,410],[255,390],[287,410],[279,380],[305,361],[269,353]],dtype=np.int32)

    isDragging = False # 드래그 여부 체크
    now=0 # 현재 도형 번호 (0:삼각형, 1:원, 2:사각형, 3:별)
    cnt=0 # 점수 카운트

    def onMouse(event, x, y, flags, param):
        global img_draw, isDragging, now, cnt
        if event == cv2.EVENT_LBUTTONDOWN:
            if 210<x<300 and 210<y<300: # 중앙 도형에 위치하는 button인지 확인
                isDragging = True
        elif event == cv2.EVENT_MOUSEMOVE:
            if isDragging:
                img_draw2 = img.copy()
                if now == 0:
                    random1 = np.array([[x,y-40],[x-50,y+40],[x+50,y+40]],dtype=np.int32)
                    cv2.polylines(img_draw2, [random1], True, (255,102,102),10)
                elif now == 1:
                    cv2.circle(img_draw2, (x,y),45,(255,102,102),10)
                elif now == 2:
                    cv2.rectangle(img_draw2, (x-40,y-40),(x+40,y+40),(255,102,102),10)
                elif now == 3:
                    random2 = np.array([[x,y-54],[x-14,y-26],[x-50,y-18],[x-24,y+1],[x-30,y+31],\
                                        [x,y+11],[x+30,y+31],[x+24,y+1],[x+50,y-18],[x+14,y-26]],dtype=np.int32)
                    cv2.polylines(img_draw2, [random2], True, (255,102,102),10) 
                cv2.imshow('GAME',img_draw2)
        elif event == cv2.EVENT_LBUTTONUP:
            if isDragging:
                isDragging = False
                # 도형 매칭 확인
                if now == 0: # 삼각형
                    if 235<x<275 and 120<y<160:
                        cnt+=1
                elif now==1: #원
                    if 350<x<390 and 235<y<275:
                        cnt+=1
                elif now==2: #사각형
                    if 120<x<160 and 235<y<275:
                        cnt+=1
                elif now==3:
                    if 235<x<275 and 360<y<400:
                        cnt+=1
                img[:100,15:150]= (170,255,255)
                cv2.putText(img, str(cnt), (50,75), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 5)
                cv2.imshow('GAME', img)
                again()

    # 게임 기본 화면            
    cv2.rectangle(img, (100,215),(180,295),(84,25,255),10) # 사각형
    cv2.circle(img, (370,255),45,(84,25,255),10) #원
    cv2.polylines(img, [pts1], True, (84,25,255),10) # 삼각형
    cv2.polylines(img, [pts2], True, (84,25,255),10) # 별모양

    # 랜덤 도형 나오게 하는 부분
    def again():
        global img, now
        img_draw1 = img.copy()
        now = np.random.randint(0,4)
        random1 = np.array([[255,215],[205,295],[305,295]],dtype=np.int32)
        random2 = np.array([[255,215],[241,243],[205,251],[231,270],[225,300],[255,280],[285,300],[279,270],[305,251],[269,243]],dtype=np.int32)
        if now == 0:
            cv2.polylines(img_draw1, [random1], True, (255,102,102),10)
        elif now == 1:
            cv2.circle(img_draw1, (255,255),45,(255,102,102),10)
        elif now == 2:
            cv2.rectangle(img_draw1, (215,215),(295,295),(255,102,102),10)
        elif now == 3:
            cv2.polylines(img_draw1, [random2], True, (255,102,102),10)
        cv2.imshow('GAME', img_draw1)        
        cv2.setMouseCallback('GAME', onMouse)

    again()    
    cv2.waitKey(0)     
    cv2.destroyAllWindows()

# GUI 구성설치

In [7]:
from tkinter import *
from PIL import Image,ImageTk
import cv2
import numpy as np
    
window = Tk()
window.title("오징어 게임")
window.geometry("800x600")
#window.resizable(width = FALSE, height = FALSE)

#opencv 이미지를 GUI에 넣기...
src = cv2.imread("./img/game_poster.jpg")
src = cv2.resize(src, (640, 400))
#src = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)
img = Image.fromarray(src)
imgtk = ImageTk.PhotoImage(image=img)

label = Label(window, image=imgtk)
label.pack(side="top")

button2 = Button(window, text="달고나 게임", command=dalgona_game)
button2.pack(side="bottom", expand=True, fill='both')

button3 = Button(window, text="숨은 그림찾기- 월리 입력하기(사용자 입력)", command=wally_input)
button3.pack(side="bottom", expand=True, fill='both')

button4 = Button(window, text="숨은 그림찾기- 월리 찾기", command=wally_find)
button4.pack(side="bottom", expand=True, fill='both')

button5 = Button(window, text="도형 맞추기", command=find_rectangle)
button5.pack(side="bottom", expand=True, fill='both')

button1 = Button(window, text="색 넣기 게임-워밍업", command=coloring_game)
button1.pack(side="bottom", expand=True, fill='both')

window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\pc\.conda\envs\hp_8_8\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-5-5b0028f31852>", line 38, in coloring_game
    r = cv2.getTrackbarPos('R', win_trackbar)
cv2.error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-_xlv4eex\opencv\modules\highgui\src\window_w32.cpp:2577: error: (-27:Null pointer) NULL window: 'trackbar' in function 'cvGetTrackbarPos'



353 117


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\pc\.conda\envs\hp_8_8\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-5-5b0028f31852>", line 37, in coloring_game
    cv2.setMouseCallback(win_name, onMouse)
cv2.error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-_xlv4eex\opencv\modules\highgui\src\window_w32.cpp:2559: error: (-27:Null pointer) NULL window: 'coloring' in function 'cvSetMouseCallback'



137 89
